In [76]:
#TRAIN_INSTANCE_TYPE = 'ml.p3.8xlarge'  # 'ml.p3.2xlarge'  # ml.m5.xlarge'
#train_instance_type = "ml.g4dn.xlarge" #'ml.p3.8xlarge'  #
TRAINING_S3_BUCKET = 'dincro-sagemaker'
AWS_ROLE = "arn:aws:iam::xxxxxxx:role/admin_sagemaker"
TAG = 'check_marks'
TRAINING_S3_FOLDER = f'training/data/{TAG}'
SRC_TRAIN_DATA="C:/WORK/wamp/www/DataFolders/check_marks/yolov4/train"
DOCKER_REPO="darknet-yolov4"
IMG_TAG="v1"
TEMP_FOLDER= "tmp_yolov4"
percentage_test = 10

train_instance_type = "ml.g4dn.2xlarge"
cfg_file = "yolov4-p6.cfg"
model_pretrain_url="https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-p6.conv.289"

#model_pretrain_url="https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-csp-swish.conv.164"
#train_instance_type = "ml.g4dn.xlarge"
#cfg_file = "yolov4-csp-x-swish.cfg"
#model_pretrain_url="https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-csp-x-swish.conv.192"

In [2]:
!aws s3 ls

2021-09-04 10:33:55 aws-cloudtrail-logs-xxxxxxx-52191963
2021-10-14 12:05:20 aws-sam-cli-managed-default-samclisourcebucket-1pyrhfewxnqpv
2021-04-16 15:47:48 cf-templates-18yu3tt8jtzgj-us-east-1
2021-04-16 14:50:33 codepipeline-us-east-1-373156545528
2021-02-26 22:15:56 dincro
2021-09-25 22:37:24 dincro-cvat
2021-06-03 10:49:41 dincro-datasets
2021-10-14 15:07:51 dincro-php-example-assets
2021-04-05 18:48:22 dincro-sagemaker
2021-08-11 21:10:32 dincro-serverless-dev
2021-08-11 21:10:22 dincro-serverless-prod
2021-08-17 15:52:00 dincro-serverless-release
2021-09-16 17:01:26 dincro-ws-dev-logs
2021-09-16 17:01:26 dincro-ws-dev-web-static
2021-08-11 22:01:24 dincro-ws-prod-logs
2021-08-11 22:01:24 dincro-ws-prod-web-static
2021-09-10 15:03:08 dincro-ws-release-logs
2021-09-10 15:03:08 dincro-ws-release-web-static
2021-07-12 20:00:57 google-annotate-dev-bucket
2021-08-11 22:30:10 google-annotate-prod-bucket
2021-04-20 09:45:53 leos3pruebas
2021-07-12 19:38:37 orch-dev-documents
2021-07-12 

invalid argument "$DOCKER_ACC/$DOCKER_REPO:$IMG_TAG" for "-t, --tag" flag: invalid reference format: repository name must be lowercase
See 'docker build --help'.
Error response from daemon: No such image: darknet-yolov4:v1


The push refers to repository [xxxxxxx.dkr.ecr.us-east-1.amazonaws.com/darknet-yolov4]


An image does not exist locally with the tag: xxxxxxx.dkr.ecr.us-east-1.amazonaws.com/darknet-yolov4


In [2]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin xxxxxxx.dkr.ecr.us-east-1.amazonaws.com

Login Succeeded


In [30]:
!docker build -t $DOCKER_REPO:$IMG_TAG .

#1 [internal] load build definition from Dockerfile
#1 sha256:a6033fe96579f582f68e779d9b1fce7e506d9f34eb0850fab0b202a77ab0d7f4
#1 transferring dockerfile: 1.34kB 0.0s done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:c3b0d8df3f9357d1be811eb94959ea9b30856e94d0c6ee40b87bfd3af29a5691
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/nvidia/cuda:11.4.2-cudnn8-devel-ubuntu20.04
#3 sha256:c72e756bcebf56362fd6aad2f92d3cf6fcdd5c4c9521f0dd009629d7ec1cec63
#3 ...

#4 [auth] nvidia/cuda:pull token for registry-1.docker.io
#4 sha256:09cc88579cacd2cfe92707543534a6ea441c4b9c409c8c70a07d894796ae17bf
#4 DONE 0.0s

#3 [internal] load metadata for docker.io/nvidia/cuda:11.4.2-cudnn8-devel-ubuntu20.04
#3 sha256:c72e756bcebf56362fd6aad2f92d3cf6fcdd5c4c9521f0dd009629d7ec1cec63
#3 DONE 11.3s

#5 [1/5] FROM docker.io/nvidia/cuda:11.4.2-cudnn8-devel-ubuntu20.04@sha256:31a4e1e133f1a181720cc70c06e69598de5881f392549bb686747c8567752ace
#5 sha256:6e99ccb5e532e6ca1

In [52]:
!docker tag $DOCKER_REPO:$IMG_TAG xxxxxxx.dkr.ecr.us-east-1.amazonaws.com/$DOCKER_REPO:$IMG_TAG

In [48]:
!aws ecr create-repository --repository-name $DOCKER_REPO --image-scanning-configuration scanOnPush=true

{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:xxxxxxx:repository/darknet-yolov4",
        "registryId": "xxxxxxx",
        "repositoryName": "darknet-yolov4",
        "repositoryUri": "xxxxxxx.dkr.ecr.us-east-1.amazonaws.com/darknet-yolov4",
        "createdAt": "2021-11-03T21:17:19-05:00",
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": true
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [53]:
!docker push xxxxxxx.dkr.ecr.us-east-1.amazonaws.com/$DOCKER_REPO:$IMG_TAG


The push refers to repository [xxxxxxx.dkr.ecr.us-east-1.amazonaws.com/darknet-yolov4]
5f70bf18a086: Preparing
cda037c9708f: Preparing
81703bfb6b28: Preparing
1698a8d506bd: Preparing
c72b17e20f65: Preparing
8fc9e4745abd: Preparing
29bd37eb2256: Preparing
7824b993090f: Preparing
080df8b22059: Preparing
bf473535509d: Preparing
a675232a9f2f: Preparing
3ac725010c75: Preparing
c74014424b6b: Preparing
4942a1abcbfa: Preparing
080df8b22059: Waiting
bf473535509d: Waiting
a675232a9f2f: Waiting
3ac725010c75: Waiting
c74014424b6b: Waiting
4942a1abcbfa: Waiting
8fc9e4745abd: Waiting
29bd37eb2256: Waiting
7824b993090f: Waiting
c72b17e20f65: Layer already exists
1698a8d506bd: Layer already exists
5f70bf18a086: Layer already exists
29bd37eb2256: Layer already exists
7824b993090f: Layer already exists
8fc9e4745abd: Layer already exists
080df8b22059: Layer already exists
bf473535509d: Layer already exists
a675232a9f2f: Layer already exists
c74014424b6b: Layer already exists
3ac725010c75: Layer already e

In [62]:
import os
import tempfile
import shutil

extracted_path = os.path.join(tempfile.gettempdir(), TEMP_FOLDER)
if os.path.exists(extracted_path):
    os.remove(extracted_path)
destination = shutil.copytree(SRC_TRAIN_DATA, extracted_path)

In [63]:
import glob
file_train = open(os.path.join(extracted_path, 'train.txt'), 'w')
file_test = open(os.path.join(extracted_path, 'test.txt'), 'w')

# Populate train.txt and test.txt
counter = 1
index_test = round(100 / percentage_test)
TRAIN_DIR = "/opt/ml/input/data/train"
current_dir= TRAIN_DIR + "/data"
for pathAndFilename in glob.iglob(os.path.join(extracted_path, "data", "*.jpg")):
    title, ext = os.path.splitext(os.path.basename(pathAndFilename))
    if counter == index_test:
        counter = 1
        file_test.write(current_dir + "/" + title + '.jpg' + "\n")
    else:
        file_train.write(current_dir + "/" + title + '.jpg' + "\n")
        counter = counter + 1

In [64]:
file_train.close()
file_test.close()

In [65]:
num_classes = sum(1 for line in open(os.path.join(extracted_path,'obj.names')))

template = """classes={num_classes}
train  = /opt/ml/input/data/train/train.txt
valid  = /opt/ml/input/data/train/test.txt
names = /opt/ml/input/data/train/obj.names
backup = /opt/ml/model/
"""
context = {
    "num_classes":num_classes
}
with  open(os.path.join(extracted_path,'obj.data'),'w') as myfile:
    myfile.write(template.format(**context))

In [66]:
import urllib.request
urllib.request.urlretrieve(model_pretrain_url, os.path.join(extracted_path,'model.weights'))


('C:\\Users\\lbola\\AppData\\Local\\Temp\\tmp_yolov4\\model.weights',
 <http.client.HTTPMessage at 0x1b6f6baeec8>)

In [71]:
!aws s3 cp $extracted_path/ s3://$TRAINING_S3_BUCKET/$TRAINING_S3_FOLDER --recursive

Completed 1.0 KiB/558.7 MiB (683 Bytes/s) with 818 file(s) remaining
upload: ..\..\..\..\..\..\..\Users\lbola\AppData\Local\Temp\tmp_yolov4\data\0385ff1c-1251-11ec-8358-ee284839698a_5.txt to s3://dincro-sagemaker/training/data/check_marks/data/0385ff1c-1251-11ec-8358-ee284839698a_5.txt
Completed 1.0 KiB/558.7 MiB (683 Bytes/s) with 817 file(s) remaining
Completed 2.1 KiB/558.7 MiB (1.4 KiB/s) with 817 file(s) remaining  
upload: ..\..\..\..\..\..\..\Users\lbola\AppData\Local\Temp\tmp_yolov4\data\021df0e4-0bf8-11ec-b6a8-fe5034e60607_2.txt to s3://dincro-sagemaker/training/data/check_marks/data/021df0e4-0bf8-11ec-b6a8-fe5034e60607_2.txt
Completed 2.1 KiB/558.7 MiB (1.4 KiB/s) with 816 file(s) remaining
Completed 3.2 KiB/558.7 MiB (2.1 KiB/s) with 816 file(s) remaining
upload: ..\..\..\..\..\..\..\Users\lbola\AppData\Local\Temp\tmp_yolov4\data\0207fdca-0cc0-11ec-ab7d-d6effb562f3f_4.txt to s3://dincro-sagemaker/training/data/check_marks/data/0207fdca-0cc0-11ec-ab7d-d6effb562f3f_4.txt
Compl

In [21]:
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role

In [26]:
#role = get_execution_role()
role = AWS_ROLE

In [77]:
estimator = Estimator(
    image_uri="xxxxxxx.dkr.ecr.us-east-1.amazonaws.com/darknet-yolov4:" + IMG_TAG,
    role=role,
    train_instance_type=train_instance_type,
    train_instance_count=1,
    train_max_run = 3 * 60 *60,
    hyperparameters={
        "data_file": "obj.data",
        "cfg_file": cfg_file
    }
)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [78]:
estimator.fit({"train": f"s3://{TRAINING_S3_BUCKET}/{TRAINING_S3_FOLDER}/"},
    wait = False)

In [ ]:
os.remove(extracted_path)